In [4]:
import pandas as pd
import datetime
from sklearn.linear_model import LinearRegression


import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

In [8]:
df = pd.read_csv('total_regular_cycle_data.csv')

In [9]:
(len(df))

50

In [10]:
df.head(10)

,M,Day,Year,Duration
0,1,25,2021,Starts
1,1,30,2021,Ends
2,2,24,2021,Starts
3,3,1,2021,Ends
4,3,26,2021,Starts
5,3,31,2021,Ends
6,4,25,2021,Starts
7,4,30,2021,Ends
8,5,23,2021,Starts
9,5,28,2021,Ends


### Start date prediction

In [12]:
# Filter only 'Starts'
df_starts = df[df['Duration'] == 'Starts'].copy()

# Rename columns to match expected names by pd.to_datetime
df_starts.rename(columns={'Year': 'year', 'M': 'month', 'Day': 'day'}, inplace=True)

# Create a datetime column
df_starts['Date'] = pd.to_datetime(df_starts[['year', 'month', 'day']])

# Sort by date
df_starts = df_starts.sort_values('Date').reset_index(drop=True)

# Drop unnecessary columns
df_starts = df_starts[['Date']]

# Convert to ordinal
df_starts['Date_Ordinal'] = df_starts['Date'].map(datetime.toordinal)

# Prepare the data for training
df_starts['Next_Date_Ordinal'] = df_starts['Date_Ordinal'].shift(-1)
df_starts.dropna(inplace=True)

X = df_starts[['Date_Ordinal']]
y = df_starts['Next_Date_Ordinal']

# Train the model
model = LinearRegression()
model.fit(X, y)

LinearRegression()

#### Saving model

In [15]:
import pickle
# Save the model to a file
with open('start_date_model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [26]:
import pickle

# Load the model from the file
with open('start_date_model.pkl', 'rb') as file:
    model = pickle.load(file)

In [29]:
# # Function to predict the next start date
# def predict_next_start_date(last_start_date):
#     # Convert date to ordinal
#     last_start_date_ordinal = last_start_date.toordinal()
#     # Create a DataFrame for the input
#     input_df = pd.DataFrame([[last_start_date_ordinal]], columns=['Date_Ordinal'])
#     # Predict the next date
#     next_start_date_ordinal = model.predict(input_df)[0]
#     # Convert ordinal back to datetime
#     next_start_date = datetime.date.fromordinal(int(next_start_date_ordinal))
#     return next_start_date

# Function to predict the next start date
def predict_next_start_date(last_start_date):
    # Convert date to ordinal
    last_start_date_ordinal = last_start_date.toordinal()
    # Create a DataFrame for the input
    input_df = pd.DataFrame([[last_start_date_ordinal]], columns=['Date_Ordinal'])
    # Predict the next date
    next_start_date_ordinal = model.predict(input_df)[0]
    # Convert ordinal back to datetime
    next_start_date = datetime.date.fromordinal(int(next_start_date_ordinal))
    return next_start_date

In [33]:
# last_start_date = datetime(2022, 1, 25)
# print('current period start date: ', last_start_date)
# for i in range(1,5):
#     pred_value = predict_next_start_date(last_start_date)
#     print('next period start date: ', pred_value)
#     last_start_date = pred_value
# from datetime import datetime, date
# last_start_date = date(2022, 1, 25)
# print('current period start date: ', last_start_date)
# for i in range(1, 5):
#     pred_value = predict_next_start_date(last_start_date)
#     print('next period start date: ', pred_value)
#     last_start_date = pred_value


# Function to predict the next start date
def predict_next_start_date(last_start_date):
    # Convert date to ordinal
    last_start_date_ordinal = last_start_date.toordinal()
    # Create a DataFrame for the input
    input_df = pd.DataFrame([[last_start_date_ordinal]], columns=['Date_Ordinal'])
    # Predict the next date
    next_start_date_ordinal = model.predict(input_df)[0]
    # Convert ordinal back to date
    next_start_date = date.fromordinal(int(next_start_date_ordinal))
    return next_start_date

# Example usage
last_start_date = date(2022, 1, 25)
print('Current period start date: ', last_start_date)
for i in range(1, 5):
    pred_value = predict_next_start_date(last_start_date)
    print('Next period start date: ', pred_value)
    last_start_date = pred_value


Current period start date:  2022-01-25
Next period start date:  2022-02-23
Next period start date:  2022-03-24
Next period start date:  2022-04-22
Next period start date:  2022-05-20


### End date prediction

In [5]:
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
df = pd.read_csv('total_regular_cycle_data.csv')
# Filter only 'Starts'
df_end = df.copy()

# Rename columns to match expected names by pd.to_datetime
df_end.rename(columns={'Year': 'year', 'M': 'month', 'Day': 'day'}, inplace=True)

# Combine Month, Day, and Year into a single date feature
df_end['Date'] = pd.to_datetime(df_end[['year', 'month', 'day']])

# Separate start and end dates
starts = df_end[df_end['Duration'] == 'Starts'].reset_index(drop=True)
ends = df_end[df_end['Duration'] == 'Ends'].reset_index(drop=True)

# Calculate the duration between start and end dates in days
starts['EndDate'] = ends['Date']
starts['Duration'] = (starts['EndDate'] - starts['Date']).dt.days

# Step 2: Feature Engineering
# Use day, month, and year as features directly
starts['Day'] = starts['Date'].dt.day
starts['Month'] = starts['Date'].dt.month
starts['Year'] = starts['Date'].dt.year

# Step 3: Model Training
# Prepare the features and labels
X = starts[['Day', 'Month', 'Year']]
y = starts['Duration']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
# scaler = StandardScaler().fit(X_train)
# X_train_scaled = scaler.transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# Train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)

print(f'Mean Absolute Error: {mae}')



Mean Absolute Error: 0.0


In [12]:
import pickle
# Save the model to a file
with open('end_date_model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [34]:
import pickle

# Load the model from the file
with open('end_date_model.pkl', 'rb') as file:
    model = pickle.load(file)

In [11]:
import pandas as pd
from datetime import datetime

def predict_end_date(year, month, day, model):
    # Create a DataFrame from input date
    new_start_data = {
        'Day': [day],
        'Month': [month],
        'Year': [year]
    }
    new_start_df_end = pd.DataFrame(new_start_data)
    
    # Scale the new data
    # new_start_scaled = scaler.transform(new_start_df_end)
    
    # Make predictions
    pred_duration = model.predict(new_start_df_end)
    
    # Convert the predicted duration back to a date
    new_start_date = datetime(year, month, day)
    pred_end_date = new_start_date + pd.Timedelta(days=pred_duration[0])
    
    return pred_end_date.strftime("%Y-%m-%d")

# Example usage:
# Assuming you have defined `model` and `scaler` elsewhere
# Replace with actual values for day, month, and year
predicted_date = predict_end_date(2021, 5, 23, model)
print(f'Predicted End Date: {predicted_date}')


Predicted End Date: 2021-05-28


### combining start and end date 

In [47]:
import pickle
from datetime import date
import pandas as pd
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# Load the model from the file
with open('start_date_model.pkl', 'rb') as file:
    model = pickle.load(file)

# Load the model from the file
with open('end_date_model.pkl', 'rb') as file:
    end_date_model = pickle.load(file)
def predict_next_start_date(last_start_date):
    # Convert date to ordinal
    last_start_date_ordinal = last_start_date.toordinal()
    # Create a DataFrame for the input
    input_df = pd.DataFrame([[last_start_date_ordinal]], columns=['Date_Ordinal'])
    # Predict the next date
    next_start_date_ordinal = model.predict(input_df)[0]
    # Convert ordinal back to date
    next_start_date = date.fromordinal(int(next_start_date_ordinal))
    return next_start_date

def predict_end_date(year, month, day, model):
    # Create a DataFrame from input date
    new_start_data = {
        'Day': [day],
        'Month': [month],
        'Year': [year]
    }
    new_start_df_end = pd.DataFrame(new_start_data)
    
    # Scale the new data
    # new_start_scaled = scaler.transform(new_start_df_end)
    
    # Make predictions
    pred_duration = model.predict(new_start_df_end)
    
    # Convert the predicted duration back to a date
    new_start_date = datetime(year, month, day)
    pred_end_date = new_start_date + pd.Timedelta(days=pred_duration[0])
    
    return pred_end_date.date()

# Example usage
last_start_date = date(2021, 3, 1)
print('Current period start date: ', last_start_date)


period_end_predicted_date = predict_end_date(last_start_date.year, last_start_date.month, last_start_date.day, end_date_model)
# print(period_end_predicted_date)
print('period end date: ', period_end_predicted_date)
print('Period duration:', period_end_predicted_date - last_start_date)

next_pred_value = predict_next_start_date(last_start_date)
print('duration in this cycle: ', next_pred_value - last_start_date)
print('next cycle start date: ', next_pred_value)



Current period start date:  2021-03-01
period end date:  2021-03-06
Period duration: 5 days, 0:00:00
duration in this cycle:  29 days, 0:00:00
next cycle start date:  2021-03-30


D:\python_enviroments\gen_env_3.11.5\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.4.1.post1 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


# All phases plus dates

In [1]:
import pickle
from datetime import date, datetime, timedelta
import pandas as pd
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# Load the model from the file
with open('start_date_model.pkl', 'rb') as file:
    model = pickle.load(file)

# Load the model from the file
with open('end_date_model.pkl', 'rb') as file:
    end_date_model = pickle.load(file)
def predict_next_start_date(last_start_date):
    # Convert date to ordinal
    last_start_date_ordinal = last_start_date.toordinal()
    # Create a DataFrame for the input
    input_df = pd.DataFrame([[last_start_date_ordinal]], columns=['Date_Ordinal'])
    # Predict the next date
    next_start_date_ordinal = model.predict(input_df)[0]
    # Convert ordinal back to date
    next_start_date = date.fromordinal(int(next_start_date_ordinal))
    return next_start_date

def predict_end_date(year, month, day, model):
    # Create a DataFrame from input date
    new_start_data = {
        'Day': [day],
        'Month': [month],
        'Year': [year]
    }
    new_start_df_end = pd.DataFrame(new_start_data)
    
    # Scale the new data
    # new_start_scaled = scaler.transform(new_start_df_end)
    
    # Make predictions
    pred_duration = model.predict(new_start_df_end)
    
    # Convert the predicted duration back to a date
    new_start_date = datetime(year, month, day)
    pred_end_date = new_start_date + pd.Timedelta(days=pred_duration[0])
    
    return pred_end_date.date()

# Example usage
last_start_date = date(2021, 5, 4)
print('Current period start date: ', last_start_date)


period_end_predicted_date = predict_end_date(last_start_date.year, last_start_date.month, last_start_date.day, end_date_model)
# print(period_end_predicted_date)
print('\nPeriod end date: ', period_end_predicted_date)
period_duration = period_end_predicted_date - last_start_date
print('\nPeriod duration:', period_duration)

next_start_pred_value = predict_next_start_date(last_start_date)
cycle_duration = next_start_pred_value - last_start_date
print('\nDuration in this cycle: ', cycle_duration)


def calculate_phases_dynamic(next_start_date, period_end_predicted_date, average_cycle, menstruation):
    if average_cycle < 22 or menstruation < 2 or menstruation >= average_cycle:
        return "Invalid input. Please enter valid average cycle and menstruation duration."
    
    # Ovulation day is typically 14 days before the end of the cycle
    ovulation_date = next_start_date - timedelta(days=14)

    # Follicular phase starts the day after menstruation ends
    follicular_start = period_end_predicted_date + timedelta(days=1)
    follicular_end = ovulation_date - timedelta(days=4)  # assuming follicular phase ends 4 days before ovulation
    
    if follicular_end < follicular_start:
        return "Invalid calculation based on provided inputs. Please adjust the input values."

    # follicular = [follicular_start.strftime('%Y-%m-%d'), follicular_end.strftime('%Y-%m-%d')]
    follicular = {'start':follicular_start.strftime('%Y-%m-%d'), 'end':follicular_end.strftime('%Y-%m-%d')}

    # Fertility window is typically 5 days, ending on the ovulation day
    fertility_start = ovulation_date - timedelta(days=3)
    fertility_end = ovulation_date + timedelta(days=2)
    # fertility = [fertility_start.strftime('%Y-%m-%d'), fertility_end.strftime('%Y-%m-%d')]
    fertility = {'start':fertility_start.strftime('%Y-%m-%d'), 'end':fertility_end.strftime('%Y-%m-%d')}

    # Luteal phase starts the day after ovulation and lasts until the end of the cycle
    luteal_start = ovulation_date + timedelta(days=3)
    luteal_end = next_start_date - timedelta(days=1)
    # luteal = [luteal_start.strftime('%Y-%m-%d'), luteal_end.strftime('%Y-%m-%d')]
    luteal = {'start':luteal_start.strftime('%Y-%m-%d'), 'end':luteal_end.strftime('%Y-%m-%d')}
    

    return {
        "follicular": follicular,
        "fertility": fertility,
        "ovulation": ovulation_date.strftime('%Y-%m-%d'),
        "luteal": luteal
    }

# menstruation = 6
phases = calculate_phases_dynamic(next_start_pred_value, period_end_predicted_date, cycle_duration.days, period_duration.days)
print('\n',phases)

# Prints the nicely formatted dictionary
# pprint.pprint(phases)
# print('next cycle start date: ', next_pred_value)


Current period start date:  2021-05-04

Period end date:  2021-05-09

Period duration: 5 days, 0:00:00

Duration in this cycle:  29 days, 0:00:00

 {'follicular': {'start': '2021-05-10', 'end': '2021-05-15'}, 'fertility': {'start': '2021-05-16', 'end': '2021-05-21'}, 'ovulation': '2021-05-19', 'luteal': {'start': '2021-05-22', 'end': '2021-06-01'}}


D:\python_enviroments\gen_env_3.11.5\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.4.1.post1 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
dict1 = {'a': 1, 'b': 2}
dict2 = {'c': 3, 'd': 4}

dict1.update(dict2)
print(dict1)
# Output: {'a': 1, 'b': 2, 'c': 3, 'd': 4}


{'a': 1, 'b': 2, 'c': 3, 'd': 4}


In [53]:
from datetime import date

my_date = date(2021, 3, 7)
formatted_date = my_date.strftime('%Y-%m-%d')

print("Formatted date:", formatted_date)


ValueError: Invalid format string

In [35]:
from datetime import datetime, timedelta
last_start_date - timedelta(days=10)
# new_date = datetime.date(2021, 1, 25) 

datetime.date(2021, 1, 15)

# Through Calculation

In [102]:
def calculate_average_cycle_length(periods):
    total_days = sum(periods)
    num_cycles = len(periods)
    average_cycle_length = total_days / num_cycles
    return average_cycle_length

# Example usage:
periods = [28, 30, 29, 31]  # list of number of days between periods for the last few cycles
average = calculate_average_cycle_length(periods)
print(f"The average menstrual cycle length is {average} days.")


The average menstrual cycle length is 29.5 days.


In [1]:
from datetime import datetime, timedelta

def calculate_next_date(input_date, average):
    # Convert the input_date string to a datetime object
    input_date = datetime.strptime(input_date, "%d-%m-%Y")
    
    # Calculate the next date by adding the average to the input_date
    next_date = input_date + timedelta(days=average)
    
    # Convert the next_date datetime object to a string
    next_date = next_date.strftime("%d-%m-%Y")
    
    return next_date


In [2]:
# print(calculate_next_date("03-01-2024", 26))  # Output: 29-01-2024
# print(calculate_next_date("01-04-2024", 26))  # Output: 27-04-2024
# print(calculate_next_date("01-05-2024", 26))  # Output: 27-04-2024
# print(calculate_next_date("07-08-2024", 26))  # Output: 27-04-2024


In [3]:
print(calculate_next_date("05-05-2024", 22))  # Output: 27-04-2024

27-05-2024


In [4]:
# last_start_date = datetime.date(2024, 5, 5)

# predict_next_start_date(last_start_date)

TypeError: descriptor 'date' for 'datetime.datetime' objects doesn't apply to a 'int' object